In [1]:
import argparse
import numpy as np
from pprint import pprint
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
from torchvision import datasets, transforms
import os
from blackbox_utils import label_to_onehot, cross_entropy_for_onehot, pixelwise_euclidean_distance
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
# 指定使用文泉驿微米黑（系统已识别的字体名称）
plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']  
# 解决负号显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False  

In [2]:
#################################################
# 轻易不要运行这个代码

# import os
# import shutil
# from PIL import Image  # 用于验证图片完整性（可选）

# # 配置路径
# source_root = "/mnt/workspace/RK-Net/data/University-Release/train/drone"
# target_dir = "/mnt/workspace/magazine/black_box/data"

# # 确保目标文件夹存在
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)
#     print(f"创建目标文件夹: {target_dir}")

# # 定义支持的图片扩展名（可根据实际情况补充）
# IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')

# # 遍历所有子文件夹，提取图片
# copied_count = 0
# for root, dirs, files in os.walk(source_root):
#     for filename in files:
#         # 检查文件扩展名是否为图片
#         if filename.lower().endswith(IMAGE_EXTENSIONS):
#             source_path = os.path.join(root, filename)
#             # 处理重名：自动添加序号（如 photo.jpg → photo_1.jpg）
#             target_path = os.path.join(target_dir, filename)
#             if os.path.exists(target_path):
#                 base_name, ext = os.path.splitext(filename)
#                 counter = 1
#                 while True:
#                     new_filename = f"{base_name}_{counter}{ext}"
#                     target_path = os.path.join(target_dir, new_filename)
#                     if not os.path.exists(target_path):
#                         break
#                     counter += 1

#             # （可选）验证图片完整性，避免复制损坏文件
#             try:
#                 with Image.open(source_path) as img:
#                     img.verify()  # 验证图片是否可正常打开
#             except (IOError, SyntaxError) as e:
#                 print(f"跳过无效图片: {source_path}（错误: {e}）")
#                 continue

#             # 复制文件（保留元数据）
#             shutil.copy2(source_path, target_path)
#             print(f"已复制: {source_path} → {target_path}")
#             copied_count += 1

# print(f"\n任务完成！共复制 {copied_count} 张图片到 {target_dir}")

In [3]:
# ------------------- 配置参数 -------------------
batchsize = 1
h, w = 384, 384
pad = 10
num_images = 3000  # 需要选取的照片数量
target_path = '/mnt/workspace/magazine/black_box/data'  # 目标图片路径
# save_dataset_path = '/mnt/workspace/magazine/black_box/blackbox_dataset.pt'  # 新数据集保存路径
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tt = transforms.ToTensor()#转发为张量
tp = transforms.ToPILImage()#转换为图片


In [4]:
# 自定义数据集（替代 ImageFolder）
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        # 只保留图片文件（过滤非图片、隐藏文件）
        self.image_paths = [
            os.path.join(image_dir, fname) 
            for fname in os.listdir(image_dir) 
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))  # 支持的图片格式
            and not fname.startswith('.')  # 过滤隐藏文件
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 加载并预处理图片
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')  # 转为 RGB
        if self.transform:
            image = self.transform(image)
        return image, 0  # 标签统一设为 0（无类别区分时）


# ------------------- 原代码替换为自定义数据集 -------------------
# 目标图片路径（直接存图片的目录）
# target_path = '/mnt/workspace/magazine/black_box/data'  

# 验证路径存在
if not os.path.exists(target_path):
    raise FileNotFoundError(f"路径不存在: {target_path}")

# 定义预处理
transform_list = [
    transforms.Resize((h, w), interpolation=3),
    transforms.Pad(pad, padding_mode='edge'),
    transforms.RandomCrop((h, w)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]
data_transform = transforms.Compose(transform_list)

# 加载自定义数据集
dataset = CustomImageDataset(target_path, transform=data_transform)

# 检查图片数量
# num_images = 5000
if len(dataset) < num_images:
    raise ValueError(f"目标路径下照片数量不足 {num_images} 张，当前仅有 {len(dataset)} 张")

selected_indices = range(num_images)  # 直接取前 num_images 张
selected_dataset = Subset(dataset, selected_indices)

# 创建数据加载器
dataloader = DataLoader(
    selected_dataset,
    batch_size=batchsize,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [5]:

# # 转换为batch格式（添加batch维度，模型通常需要[batch, c, h, w]格式）
# preprocessed_img_torch = tt(preprocessed_img)
# preprocessed_img_torch = preprocessed_img_torch.unsqueeze(0)  # 形状从[c, h, w]变为[1, c, h, w]
# label_image_torch = torch.tensor([label_image])  # 标签添加batch维度



In [6]:
# from model_white_attack import ft_net
# pretrained_path = '/mnt/workspace/RK-Net/RK-Net-main/model/pretrain/net_359.pth'  # 绝对路径
# # RK-Net/RK-Net-main/model/pretrain/net_359.pth
# model = ft_net(751).to(device)

# # 加载预训练权重（strict=False允许部分参数不匹配，适用于迁移学习）
# pretrained_state = torch.load(pretrained_path, map_location=torch.device('cpu'))
# model.load_state_dict(pretrained_state, strict=False)
# print("Loaded pretrained model from:", pretrained_path)

| 索引 | 模块名称                          | 说明                          |
|------|-----------------------------------|-------------------------------|
| 0    | `model_ft.conv1`                  | 第1层：初始卷积层             |
| 1    | `model_ft.bn1`                    | 第2层：批归一化层             |
| 2    | `model_ft.relu`                   | 第3层：ReLU激活函数           |
| 3    | `self.usam_1`                     | 第4层：USAM注意力模块（第一个）|
| 4    | `model_ft.maxpool`                | 第5层：最大池化层             |
| 5    | `model_ft.layer1`                 | 第6层：ResNet第1个残差块      |
| 6    | `self.usam_2`                     | 第7层：USAM注意力模块（第二个）|
| 7    | `model_ft.layer2`                 | 第8层：ResNet第2个残差块      |
| 8    | `model_ft.layer3`                 | 第9层：ResNet第3个残差块      |
| 9    | `model_ft.layer4`                 | 第10层：ResNet第4个残差块     |
| 10   | `model_ft.avgpool2`               | 第11层：全局平均池化层        |
| 11   | `lambda x: x.view(x.size(0), x.size(1))` | 第12层：特征展平操作（将空间维度展平为特征向量） |

In [7]:


class VGG9(nn.Module):
    def __init__(self, num_classes, sp=4, in_channels=3, fig_size=384):####sp为切割点
        assert sp<=17 and sp>0
        super(VGG9, self).__init__()
        cfg = [32, 64, 128, 128, 256, 256, 512, 512]
        self.conv1 = nn.Conv2d(in_channels,  cfg[0], 3, 1, 1)
        self.conv2 = nn.Conv2d(cfg[0], cfg[1], 3, 1, 1)
        self.conv3 = nn.Conv2d(cfg[1], cfg[2], 3, 1, 1)
        self.conv4 = nn.Conv2d(cfg[2], cfg[3], 3, 1, 1)
        self.conv5 = nn.Conv2d(cfg[3], cfg[4], 3, 1, 1)
        self.conv6 = nn.Conv2d(cfg[4], cfg[5], 3, 1, 1)
        self.fc1   = nn.Linear(int((fig_size/8)**2)*cfg[5], cfg[6])
        self.fc2   = nn.Linear(cfg[6], cfg[7])
        self.fc3   = nn.Linear(cfg[7], num_classes)
        self.total_module_list = [self.conv1, self.conv2, nn.ReLU(), nn.MaxPool2d(2, 2), 
                                self.conv3, self.conv4, nn.ReLU(), nn.MaxPool2d(2, 2),
                                self.conv5, self.conv6, nn.ReLU(), nn.MaxPool2d(2, 2),
                               nn.Flatten(1), self.fc1, nn.ReLU(), self.fc2, nn.ReLU(), self.fc3]
        self.f = nn.Sequential(*self.total_module_list[0:sp])
        self.g = nn.Sequential(*self.total_module_list[sp:])

    def forward(self, x):
        x_1 = self.f(x)
        x_cut = x_1.detach().clone()
        x_out = self.g(x_1)
        return x_out, x_cut

net = VGG9(num_classes=751).to(device)#模型名字


In [8]:
# ------------------- 生成新训练集（简化版，无分块） -------------------
new_dataset = []
save_dataset_path = '/mnt/workspace/magazine/black_box/blackbox_dataset.pt'
# with torch.no_grad():
#     for i, (img_tensor, _) in enumerate(tqdm(dataloader, desc="生成新数据集")):
#         # 模型推理（确保所有计算在GPU）
#         img_tensor = img_tensor.to(device)
#         model_output = net.f(img_tensor)
        
#         # 获取原始图像路径（绝对路径，确保可靠）
#         original_path = selected_dataset.dataset.image_paths[selected_indices[i]]
        
#         # 保存：模型输出移到CPU（释放GPU内存），路径保持绝对
#         new_dataset.append((model_output.cpu(), img_tensor.cpu()))
        
#         # # 每100步清理GPU缓存（可选）
#         # if i % 100 == 0:
#         #     torch.cuda.empty_cache()

# # 直接保存完整数据集（避免分块合并错误）
# torch.save(new_dataset, save_dataset_path)
# print(f"✅ 已保存完整数据集到: {save_dataset_path}")

In [9]:
# # save_dataset_path = '/mnt/workspace/magazine/black_box/blackbox_dataset.pt'
# # ------------------- 验证新数据集 -------------------
# # 加载并验证新数据集
# loaded_dataset = torch.load(save_dataset_path)
# print(f"加载验证: 数据集包含 {len(loaded_dataset)} 个样本")

# # 示例：查看第一个样本
# sample_output, sample_path = loaded_dataset[0]
# print(f"样本输出形状: {sample_output.shape}")
# print(f"对应的原始图像路径: {sample_path}")

In [10]:
class blackbox_attack(nn.Module):
    def __init__(self, num_classes, sp=4, in_channels=64, fig_size=192):
        # 确保 sp 范围有效
        assert 0 < sp <= 17, "sp 需满足 0 < sp <= 17"
        super(blackbox_attack, self).__init__()  # 修正继承，原为错误的 VGG9
        
        # 网络配置（通道数序列）
        cfg = [32, 64, 128, 128, 256, 256, 512, 512]
        
        # ------------------- 新增：上采样转置卷积层 -------------------
        # 目标：将 192×192 放大到 384×384（尺寸翻倍）
        self.upsample = nn.ConvTranspose2d(
            in_channels=cfg[1],    # 输入通道数（与 conv1 输出匹配）
            out_channels=cfg[1],   # 输出通道数（保持与输入一致）
            kernel_size=3,         # 卷积核大小
            stride=2,              # 步长=2 → 尺寸×2
            padding=1,             # 填充
            output_padding=1,      # 确保输出尺寸精准翻倍（192→384）
            bias=False             # 配合 BatchNorm 更稳定
        )
        # 可选：添加 BatchNorm 和 ReLU 增强非线性（根据需求调整）
        self.upsample_bn = nn.BatchNorm2d(cfg[1])
        self.upsample_relu = nn.ReLU(inplace=True)
        
        # ------------------- 原有卷积层 -------------------
        self.conv1 = nn.Conv2d(cfg[1], cfg[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(cfg[0], in_channels, kernel_size=3, stride=1, padding=1)
        
        # ------------------- 构建模块列表（插入上采样流程） -------------------
        self.total_module_list = [
            # 插入上采样流程：64×192×192 → 64×384×384
            self.upsample,        
            self.upsample_bn,     
            self.upsample_relu,   
            self.conv1,            
            nn.ReLU(inplace=True), 
            
            self.conv2,            
        ]

    def forward(self, x):
        # 按模块列表顺序执行
        for module in self.total_module_list:
            x = module(x)
        return x

In [15]:
attack_dataset = torch.load(save_dataset_path)

# 创建数据加载器（全部作为训练集，不划分验证集）
attack_train_loader = DataLoader(attack_dataset, batch_size=32, shuffle=True, num_workers=2)

# ------------------- 初始化模型、损失函数和优化器 -------------------
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
blackbox_net = blackbox_attack(num_classes=751).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(blackbox_net.parameters(), lr=0.001)
epochs = 20
history_attack = []
# ------------------- 训练循环 -------------------
for epoch in range(epochs):
    # 训练阶段
    blackbox_net.train()
    train_loss = 0.0
    for model_output, preprocessed_tensor in tqdm(attack_train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        print(preprocessed_tensor[0].shape)
        print(model_output.shape)
        model_output = model_output.to(device)
        preprocessed_tensor = preprocessed_tensor.to(device)
        
        # 前向传播
        outputs = blackbox_net(model_output)
        loss = criterion(outputs, preprocessed_tensor)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # train_loss += loss.item() * model_output.size(0)
    history_attack.append(tp(outputs[0].cpu()))
    print(f"epoch {epoch} loss value:{loss}")
    
    
plt.figure(figsize=(12, 8))
for i in range(20):
    plt.subplot(2, 10, i + 1)
    plt.imshow(history[i])
    plt.title("虚拟图片展示（iter=%d)" % (i))
    plt.axis('off')


# 保存最终模型
torch.save(blackbox_net.state_dict(), 'w_attack.pth')
print("训练完成！最终模型已保存至 w_attack.pth")

Epoch 1/20 [Train]:   0%|          | 0/94 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'shape'

In [ ]:
# print(dummy_loss)